# Aurora — SSOT + XAI/TCA/Risk Analysis with Optuna

This Colab notebook loads SSOT config, ingests `*.jsonl.gz` logs, computes metrics, visualizes distributions, manages local profiles, runs multi-objective Optuna (maximize EΠ_after_TCA, minimize CVaR95), renders a Pareto frontier, and performs a simple OOS re-evaluation of top configs.


ІНСТРУКЦІЇ (Setup):\n- Натисни ▶ у наступній комірці, щоб встановити бібліотеки.\n- Дочекайся зеленої галочки, потім переходь далі.\n

In [ ]:
# ІНСТРУКЦІЯ: Натисни кнопку ▶ зліва від цієї комірки.
# Вона встановить потрібні бібліотеки у Colab.
# Просто натисни і зачекай, поки з'явиться зелена галочка ліворуч.
# Якщо не у Colab, комірка теж безпечна.
# Після завершення переходь до наступної комірки.
# Setup - install extras only if running in Colab
import sys, subprocess, os
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip -q install pandas numpy seaborn matplotlib plotly optuna tomli ipywidgets

import json, gzip, math, io
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

# Plotly is optional; use matplotlib fallback if unavailable
try:
    import plotly.express as px
except Exception:
    px = None

try:
    import optuna
except Exception as e:
    raise RuntimeError('Optuna is required. Please install optuna.') from e

# Widgets
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
except Exception:
    widgets = None


In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб знайти кореневу папку проекту.
# Якщо з'явиться запит на шлях - встав повний шлях до папки, де є папка 'configs'.
# У Colab зазвичай шлях вже правильний. Після виконання побачиш текст 'Using ROOT = ...'.
# Environment - detect repository root (expects 'configs/' to exist)
ROOT = Path.cwd()
if not (ROOT / 'configs').exists():
    # If running outside the repo root in Colab, let user set a custom path
    print('Repository root not found in current directory.')
    ROOT = Path(input('Enter absolute path to repo root (containing configs/): ').strip())
assert (ROOT / 'configs').exists(), 'configs/ directory not found at ROOT'
REPORTS = ROOT / 'reports'
REPORTS.mkdir(parents=True, exist_ok=True)
print('Using ROOT=', ROOT)


## Paths (Google Drive)

ІНСТРУКЦІЇ:
- У Colab дані і конфіги мають бути у `/content/drive/MyDrive`.
- Натисни ▶ у наступній комірці — ми зафіксуємо ROOT на MyDrive.


In [ ]:
# Force ROOT to MyDrive paths in Colab
if IN_COLAB:
    try:
        from google.colab import drive as _drive
        _drive.mount('/content/drive', force_remount=False)
    except Exception as _e:
        print('Drive mount warning:', _e)
    from pathlib import Path as _P
    ROOT = _P('/content/drive/MyDrive')
    REPORTS = ROOT / 'reports'
    REPORTS.mkdir(parents=True, exist_ok=True)
    print('Using ROOT=', ROOT)


In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб завантажити конфігурацію (SSOT).
# Після виконання побачиш перелік секцій конфігу у вигляді таблиці.
# SSOT loading - configs/schema.json + configs/default.toml
# Prefer project loader; fallback to raw TOML if import fails.
def _toml_load_bytes(b):
    try:
        import tomllib
        return tomllib.loads(b.decode('utf-8') if isinstance(b, (bytes, bytearray)) else b)
    except Exception:
        import tomli
        return tomli.loads(b.decode('utf-8') if isinstance(b, (bytes, bytearray)) else b)

def deep_get(d, path, default=None):
    cur = d
    for p in path.split('.'):
        if not isinstance(cur, dict) or p not in cur:
            return default
        cur = cur[p]
    return cur

def deep_merge(a: dict, b: dict) -> dict:
    out = json.loads(json.dumps(a))
    for k, v in (b or {}).items():
        if isinstance(v, dict) and isinstance(out.get(k), dict):
            out[k] = deep_merge(out[k], v)
        else:
            out[k] = v
    return out

def normalize_profile_flat_keys(profile: dict) -> dict:
    mapping = {
        'execution_sla_max_latency_ms': ('execution','sla','max_latency_ms'),
        'replay_chunk_minutes': ('replay','chunk_minutes'),
        'sizing_max_position_usd': ('sizing','limits','max_notional_usd'),
        'sizing_leverage': ('sizing','limits','leverage_max'),
        'universe_top_n': ('universe','ranking','top_n'),
        'universe_spread_bps_limit': ('execution','router','spread_limit_bps'),
        'reward_ttl_minutes': ('reward','ttl_minutes'),
        'xai_sample_every': ('xai','decision_log','sample_every'),
        'xai_level': ('xai','decision_log','level'),
        'xai_sig': ('xai','decision_log','enabled'),
        'leadlag_enable': ('leadlag','enable'),
        'leadlag_lag_grid': ('leadlag','lag_grid'),
    }
    out = {}
    for k, v in profile.items():
        if k in mapping:
            parts = mapping[k]
            cur = out
            for p in parts[:-1]:
                cur = cur.setdefault(p, {})
            cur[parts[-1]] = v
        else:
            out[k] = v
    return out

def load_ssot(config_path=None, schema_path=None):
    config_path = config_path or (ROOT / 'configs' / 'default.toml')
    schema_path = schema_path or (ROOT / 'configs' / 'schema.json')
    try:
        sys.path.insert(0, str(ROOT))
        from core.config.loader import load_config
        cfg = load_config(config_path=config_path, schema_path=schema_path, enable_watcher=False)
        cfgd = cfg.as_dict()
    except Exception:
        cfgd = _toml_load_bytes(Path(config_path).read_bytes())
        # Schema defaults best-effort: we won't re-validate here.
    return cfgd

CFG = load_ssot()
sections = {
    'sizing': CFG.get('sizing', {}),
    'risk': CFG.get('risk', {}),
    'execution': CFG.get('execution', {}),
    'reward': CFG.get('reward', {}),
    'tca': CFG.get('tca', {}),
    'features': CFG.get('features', {}),
    'universe': CFG.get('universe', {}),
    'sim_local': CFG.get('order_sink', {}).get('sim_local', {}),
}
print('Loaded SSOT sections:', ', '.join(sections.keys()))
pd.DataFrame({k:[str(type(v).__name__), len(v) if hasattr(v, 'items') else ''] for k,v in sections.items()}, index=['type','size'])


## Load Logs (*.jsonl.gz)

ІНСТРУКЦІЇ:
- Поклади файли з логами до `/content/drive/MyDrive` (напр. `datasets/`).
- Натисни ▶ у наступній комірці — спочатку пошукаємо `*.jsonl.gz` у MyDrive.
- Якщо файлів не знайдено — з’явиться вікно, де можна обрати `*.jsonl.gz` вручну.
- Після виконання шукай текст: `Loaded rows: N`.


In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб завантажити логи.
# У Colab з'явиться вікно вибору файлів - обери свої *.jsonl.gz.
# Коли все пройде, побачиш напис 'Loaded rows: N'. Якщо N=0 - дані не знайдено, це не помилка.
# In Colab, use the upload widget; otherwise specify local paths.
LOG_FILES = []  # you can set to list of absolute paths
if IN_COLAB and not LOG_FILES:
    try:
        from google.colab import files
        uploaded = files.upload()
        LOG_FILES = [Path(k) for k in uploaded.keys()]
    except Exception:
        pass

def read_jsonl_any(path: Path) -> list:
    buf = ''
    try:
        if str(path).endswith('.gz'):
            with gzip.open(path, 'rt', encoding='utf-8') as f:
                buf = f.read()
        else:
            buf = Path(path).read_text(encoding='utf-8')
    except Exception:
        return []
    out = []
    for line in buf.splitlines():
        try:
            out.append(json.loads(line))
        except Exception:
            pass
    return out

def load_logs_to_df(paths):
    records = []
    for p in paths:
        records.extend(read_jsonl_any(Path(p)))
    if not records:
        return pd.DataFrame()
    return pd.json_normalize(records)

df_logs = load_logs_to_df(LOG_FILES)
print('Loaded rows:', len(df_logs))
display(df_logs.head(3))


## Parse and Compute Metrics

ІНСТРУКЦІЇ:
- Натисни ▶ у наступній комірці, щоб порахувати метрики.
- Результат буде у вигляді JSON з блоками: order_level, session_level, tca, risk.
- Якщо в логах бракує деяких полів — частина значень може бути NaN (це нормально).


In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб порахувати метрики.
# Зачекай, і внизу побачиш JSON з результатами. Якщо деякі значення NaN - це нормально при малих даних.
# Helpers for lifecycle latencies (nearest-rank percentiles)
def _percentiles(values, qs=[50,95,99]):
    if not len(values):
        return {q:0.0 for q in qs}
    arr = sorted(values)
    n = len(arr)
    out = {}
    for q in qs:
        k = max(1, min(n, int((q/100.0)*n + 0.9999)))
        out[q] = float(arr[k-1])
    return out

def correlate_latencies(df: pd.DataFrame):
    # Expect columns: 'cid'/'oid', 'ts_ns' or 'ts_ms', 'event_code' or similar
    if df.empty:
        return {'submit_ack':{'p50':0,'p95':0,'p99':0}, 'ack_done':{'p50':0,'p95':0,'p99':0}}
    by = {}
    def norm_ts_ns(row):
        if pd.notna(row.get('ts_ns')):
            try:
                return int(row['ts_ns'])
            except Exception:
                pass
        if pd.notna(row.get('ts_ms')):
            return int(float(row['ts_ms'])*1e6)
        if pd.notna(row.get('ts')):
            t = float(row['ts'])
            if t >= 1e12:
                return int(t*1e6)
            return int(t*1e9)
        return None
    for _, r in df.iterrows():
        cid = r.get('cid')
        oid = r.get('oid')
        key = str(cid or oid)
        if not key:
            continue
        st = by.setdefault(key, {'submit':None,'ack':None,'done':None})
        ts = norm_ts_ns(r)
        et = str(r.get('event_code') or r.get('type') or r.get('event') or '').upper().replace('.', '_')
        if et.endswith('ORDER_SUBMIT') or et=='ORDER_SUBMIT':
            st['submit'] = ts
        elif et.endswith('ORDER_ACK') or et=='ORDER_ACK':
            st['ack'] = ts
        elif et.endswith('ORDER_CANCEL') or et=='ORDER_CANCEL' or et.endswith('ORDER_REJECT') or et.endswith('ORDER_EXPIRE') or et.endswith('ORDER_FILL'):
            st['done'] = ts
    submit_ack = []
    ack_done = []
    for st in by.values():
        if st['submit'] and st['ack']:
            submit_ack.append((st['ack']-st['submit'])/1e6)
        if st['ack'] and st['done']:
            ack_done.append((st['done']-st['ack'])/1e6)
    p1 = _percentiles(submit_ack)
    p2 = _percentiles(ack_done)
    return {'submit_ack':{'p50':p1[50],'p95':p1[95],'p99':p1[99]}, 'ack_done':{'p50':p2[50],'p95':p2[95],'p99':p2[99]}}

def compute_metrics(df: pd.DataFrame):
    out = {}
    if df.empty:
        return out
    # Order-level ratios
    qty = df.get('order_qty') or df.get('qty')
    filled = df.get('filled_qty') or df.get('fill_qty')
    fill_ratio = None
    try:
        q = pd.to_numeric(qty, errors='coerce')
        f = pd.to_numeric(filled, errors='coerce')
        fill_ratio = (f / q).clip(0,1)
    except Exception:
        pass
    # Maker/Taker flags (best-effort)
    side = (df.get('liquidity') or df.get('taker_maker') or df.get('context.liquidity'))
    maker_mask = side.astype(str).str.lower().str.contains('maker') if side is not None else pd.Series([False]*len(df))
    maker_fill_ratio = None
    if fill_ratio is not None:
        maker_fill_ratio = fill_ratio[maker_mask].mean() if maker_mask.any() else np.nan
    # Cancel ratio
    code = (df.get('event_code') or df.get('status') or df.get('final'))
    is_cancel = code.astype(str).str.upper().str.contains('CANCEL') if code is not None else pd.Series([False]*len(df))
    cancel_ratio = float(is_cancel.mean()) if len(is_cancel)>0 else np.nan
    # Latencies from lifecycle events (if a separate events log exists, re-run with that DataFrame)
    lat = correlate_latencies(df)
    # Slippage/IS
    slip = pd.to_numeric(df.get('slippage_bps'), errors='coerce') if 'slippage_bps' in df.columns else None
    is_bps = pd.to_numeric(df.get('is_bps'), errors='coerce') if 'is_bps' in df.columns else None
    # Session-level
    trades = int((code.astype(str).str.upper().str.contains('FILL').sum()) if code is not None else len(df))
    # Trades per minute heuristic
    tms = pd.to_numeric(df.get('ts_ms'), errors='coerce') if 'ts_ms' in df.columns else None
    tsec = pd.to_numeric(df.get('ts'), errors='coerce') if 'ts' in df.columns else None
    if tms is not None and tms.notna().any():
        dur_min = (tms.max()-tms.min())/60000.0
    elif tsec is not None and tsec.notna().any():
        dur_min = (tsec.max()-tsec.min())/60.0
    else:
        dur_min = np.nan
    trades_per_min = (trades/max(1e-9, dur_min)) if not math.isnan(dur_min) else np.nan
    # Sharpe_daily proxy from IS series (bps)
    sharpe_daily = np.nan
    ref = is_bps if is_bps is not None and is_bps.notna().any() else ( -slip if slip is not None else None )
    if ref is not None:
        s = ref.dropna()
        if len(s) >= 2:
            mu, sd = s.mean(), s.std(ddof=1)
            sharpe_daily = (mu / (sd+1e-9)) * np.sqrt(1440.0)  # 1-min bars proxy
    # HitRate
    hit_rate = float((ref>0).mean()) if ref is not None and len(ref.dropna())>0 else np.nan
    # kappa (bps/ms): mean(IS)/mean(lat_ms)
    kappa = np.nan
    lat_ack = lat['submit_ack']['p50'] if lat else np.nan
    if ref is not None and not math.isnan(lat_ack) and lat_ack>0:
        kappa = float(ref.dropna().mean())/float(lat_ack)
    # TCA components (if present)
    comp = {}
    for k in ['is_bps','spread_bps','latency_bps','adverse_bps','impact_bps','fees_bps']:
        if k in df.columns:
            comp[k] = float(pd.to_numeric(df[k], errors='coerce').dropna().mean())
    # Risk metrics: CVaR95 on reference series (downside)
    cvar95 = np.nan
    if ref is not None and len(ref.dropna())>=20:
        x = ref.dropna().values.astype(float)
        q = np.quantile(x, 0.05)
        tail = x[x<=q]
        cvar95 = -float(tail.mean())  # positive number for downside magnitude
    # deny-rate, SLA breach
    deny = df.get('reason_code') if 'reason_code' in df.columns else df.get('deny_reason')
    deny_rate = float(pd.notna(deny).mean()) if deny is not None else 0.0
    sla_max = deep_get(CFG, 'execution.sla.max_latency_ms', 25)
    sla_breach = float((pd.Series([lat_ack]) > float(sla_max)).mean()) if not math.isnan(lat_ack) else np.nan
    # Pack
    out = {
        'order_level': {
            'fill_ratio_mean': float(fill_ratio.mean()) if fill_ratio is not None else np.nan,
            'maker_fill_ratio': float(maker_fill_ratio) if maker_fill_ratio is not None else np.nan,
            'cancel_ratio': float(cancel_ratio),
            'latency_ms': lat,
            'slip_bps_mean': float(slip.mean()) if slip is not None else np.nan,
        },
        'session_level': {
            'trades_per_min': float(trades_per_min),
            'sharpe_daily': float(sharpe_daily) if not math.isnan(sharpe_daily) else np.nan,
            'hit_rate': float(hit_rate) if not math.isnan(hit_rate) else np.nan,
            'kappa_bps_per_ms': float(kappa) if not math.isnan(kappa) else np.nan,
        },
        'tca': {
            'IS_bps': float(comp.get('is_bps', np.nan)),
            'Spread_bps': float(comp.get('spread_bps', np.nan)),
            'Latency_bps': float(comp.get('latency_bps', np.nan)),
            'Adverse_bps': float(comp.get('adverse_bps', np.nan)),
            'Impact_bps': float(comp.get('impact_bps', np.nan)),
            'Fees_bps': float(comp.get('fees_bps', np.nan)),
        },
        'risk': {
            'CVaR95': float(cvar95) if not math.isnan(cvar95) else np.nan,
            'deny_rate': float(deny_rate),
            'SLA_breach': float(sla_breach) if not math.isnan(sla_breach) else np.nan,
        },
    }
    return out, ref

metrics, ref_series = compute_metrics(df_logs)
print(json.dumps(metrics, indent=2))


## Visualizations

ІНСТРУКЦІЇ:
- Натисни ▶ у наступній комірці — побачиш графіки: IS/слiпідж, fill_ratio, deny-коди, CVaR хвіст.
- Якщо даних мало — графіки можуть бути порожні або короткі.


In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб намалювати графіки.
# Після виконання прокрути вниз - побачиш гістограми та стовпчики.
# IS distribution, fill_ratio (if inferable), deny codes, CVaR tail
fig, axes = plt.subplots(1, 3, figsize=(16,4))
if 'is_bps' in df_logs.columns:
    sns.histplot(pd.to_numeric(df_logs['is_bps'], errors='coerce').dropna(), bins=50, ax=axes[0])
    axes[0].set_title('IS_bps distribution')
else:
    sns.histplot(pd.to_numeric(df_logs.get('slippage_bps', pd.Series(dtype=float)), errors='coerce').dropna(), bins=50, ax=axes[0])
    axes[0].set_title('slip_bps distribution')
# Fill ratio proxy
qty = pd.to_numeric(df_logs.get('order_qty', np.nan), errors='coerce')
fqty = pd.to_numeric(df_logs.get('filled_qty', np.nan), errors='coerce')
fr = (fqty/qty).clip(0,1) if isinstance(qty, pd.Series) and isinstance(fqty, pd.Series) else pd.Series(dtype=float)
sns.histplot(fr.dropna(), bins=50, ax=axes[1])
axes[1].set_title('fill_ratio distribution')
# Deny codes
deny = df_logs.get('reason_code') if 'reason_code' in df_logs.columns else df_logs.get('deny_reason')
if deny is not None:
    top = deny.astype(str).value_counts().head(10)
    sns.barplot(x=top.values, y=top.index, ax=axes[2])
    axes[2].set_title('Top deny codes')
plt.tight_layout()
plt.show()

# CVaR tail plot
if ref_series is not None and isinstance(ref_series, pd.Series) and len(ref_series.dropna())>10:
    x = ref_series.dropna().sort_values()
    n = len(x)
    cut = int(max(1, math.floor(0.05*n)))
    tail = x.iloc[:cut]
    plt.figure(figsize=(6,3))
    sns.histplot(tail, bins=30)
    plt.title('CVaR95 tail (worst 5%)')
    plt.show()


## Profiles (local_low/mid/high) and Effective Config

ІНСТРУКЦІЇ:
- Натисни ▶ у наступній комірці — згенеруємо ефективний конфіг для обраного профілю.
- Файл буде збережено в `/content/drive/MyDrive/reports/effect_<profile>.toml`.
- Далі відкрий секцію 'Interactive Controls' для зміни параметрів.


In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб підготувати профілі і зібрати ефективний конфіг.
# Після цього переходь до наступної секції 'Interactive Controls'. Також буде збережено effect_<profile>.toml у reports.
profiles = CFG.get('profile', {})
print('Available profiles:', list(profiles.keys()))
SELECTED_PROFILE = 'local_mid'  # change as needed
prof_raw = profiles.get(SELECTED_PROFILE, {})
prof = normalize_profile_flat_keys(prof_raw)
effective_cfg = deep_merge(CFG, prof)
print('Effective config overlayed with profile =', SELECTED_PROFILE)
# Helper: minimal TOML dumper
def dump_toml(d: dict, prefix: str = '') -> str:
    lines = []
    simple = {k:v for k,v in d.items() if not isinstance(v, dict)}
    nested = {k:v for k,v in d.items() if isinstance(v, dict)}
    def _fmt(v):
        if isinstance(v, str): return f'"{v}"'
        if isinstance(v, bool): return str(v).lower()
        return json.dumps(v) if isinstance(v, (list, tuple)) else str(v)
    for k, v in simple.items():
        lines.append(f"{k} = {_fmt(v)}")
    for k, v in nested.items():
        table = f"{prefix}{k}" if not prefix else f"{prefix}.{k}"
        lines.append('')
        lines.append(f"[{table}]")
        lines.extend(dump_toml(v, table).splitlines())
    return '
'.join(lines)

# Preview a few key sections
for sec in ['sizing','risk','execution','tca','features','universe','order_sink']:
    if sec in effective_cfg:
        print(f'[{sec}] keys:', list(effective_cfg[sec].keys())[:10])

# Optionally write a profile-effective TOML to reports (for review)
eff_text = dump_toml(effective_cfg)
(REPORTS / f'effect_{SELECTED_PROFILE}.toml').write_text(eff_text, encoding='utf-8')
print('Wrote', REPORTS / f'effect_{SELECTED_PROFILE}.toml')
# Preview key parameters requested: sizing max_position_usd, universe top_n/spread_bps_limit, reward TTL
print('sizing.limits.max_notional_usd =', deep_get(effective_cfg, 'sizing.limits.max_notional_usd'))
print('universe.ranking.top_n =', deep_get(effective_cfg, 'universe.ranking.top_n', 'N/A'))
print('execution.router.spread_limit_bps =', deep_get(effective_cfg, 'execution.router.spread_limit_bps', 'N/A'))
print('reward.ttl_minutes =', deep_get(effective_cfg, 'reward.ttl_minutes', 'N/A'))


## Interactive Controls

ІНСТРУКЦІЇ:
- Обери профіль (local_low/mid/high) у випадаючому меню.
- Посунь слайдери: top_n, TTL (min), max_notional, spread_bps.
- Заповни числа: latency_ms, leverage.
- Натисни 'Apply to Config' — конфіг оновиться та запишеться до reports/.


In [ ]:
# ІНСТРУКЦІЯ: 1) Обери профіль у випадаючому списку. 2) Посунь слайдери і зміни числа як хочеш.
# 3) Натисни кнопку 'Apply to Config'. 4) Читай повідомлення нижче - там будуть обрані значення.
# Dropdown for profile; sliders and numeric inputs for core knobs.
if widgets is None:
    print('ipywidgets not available. Install ipywidgets to use controls.')
else:
    prof_opts = list(CFG.get('profile', {}).keys()) or ['local_low','local_mid','local_high']
    prof_dd = widgets.Dropdown(options=prof_opts, value=SELECTED_PROFILE if 'SELECTED_PROFILE' in globals() else (prof_opts[0] if prof_opts else None), description='Profile')
    # Defaults from current effective_cfg or sensible fallbacks
    def _dv(path, dv):
        return deep_get(effective_cfg if 'effective_cfg' in globals() else CFG, path, dv)
    top_n = int(_dv('universe.ranking.top_n', 10) or 10)
    ttl = int(_dv('reward.ttl_minutes', 120) or 120)
    max_notional = float(_dv('sizing.limits.max_notional_usd', 5000.0) or 5000.0)
    spread_bps = float(_dv('execution.router.spread_limit_bps', 30.0) or 30.0)
    latency_ms = int(_dv('execution.sla.max_latency_ms', 25) or 25)
    leverage = float(_dv('sizing.limits.leverage_max', 2.0) or 2.0)
    s_topn = widgets.IntSlider(value=top_n, min=1, max=100, step=1, description='top_n')
    s_ttl = widgets.IntSlider(value=ttl, min=5, max=720, step=5, description='TTL (min)')
    s_maxn = widgets.FloatSlider(value=max_notional, min=100.0, max=50000.0, step=100.0, readout_format='.0f', description='max_notional')
    s_spread = widgets.FloatSlider(value=spread_bps, min=1.0, max=200.0, step=1.0, description='spread_bps')
    i_latency = widgets.BoundedIntText(value=latency_ms, min=1, max=1000, step=1, description='latency_ms')
    f_lev = widgets.BoundedFloatText(value=leverage, min=1.0, max=25.0, step=0.1, description='leverage')
    apply_btn = widgets.Button(description='Apply to Config', button_style='success')
    out = widgets.Output()

    def on_apply(_):
        global SELECTED_PROFILE, effective_cfg
        SELECTED_PROFILE = prof_dd.value
        prof_raw = CFG.get('profile', {}).get(SELECTED_PROFILE, {})
        prof_norm = normalize_profile_flat_keys(prof_raw)
        # Overlay profile first
        cfg1 = deep_merge(CFG, prof_norm)
        # Then overlay widget values
        overlay = {
            'universe': {'ranking': {'top_n': int(s_topn.value)}},
            'reward': {'ttl_minutes': int(s_ttl.value)},
            'sizing': {'limits': {'max_notional_usd': float(s_maxn.value), 'leverage_max': float(f_lev.value)}},
            'execution': {'router': {'spread_limit_bps': float(s_spread.value)}, 'sla': {'max_latency_ms': int(i_latency.value)}},
        }
        effective_cfg = deep_merge(cfg1, overlay)
        # Persist
        (REPORTS / f'effect_{SELECTED_PROFILE}.toml').write_text(dump_toml(effective_cfg), encoding='utf-8')
        with out:
            clear_output(wait=True)
            print('Applied. Effective config saved to', REPORTS / f'effect_{SELECTED_PROFILE}.toml')
            print('top_n =', deep_get(effective_cfg, 'universe.ranking.top_n'))
            print('reward.ttl_minutes =', deep_get(effective_cfg, 'reward.ttl_minutes'))
            print('max_notional_usd =', deep_get(effective_cfg, 'sizing.limits.max_notional_usd'))
            print('spread_limit_bps =', deep_get(effective_cfg, 'execution.router.spread_limit_bps'))
            print('latency_ms =', deep_get(effective_cfg, 'execution.sla.max_latency_ms'))
            print('leverage_max =', deep_get(effective_cfg, 'sizing.limits.leverage_max'))

    apply_btn.on_click(on_apply)
    display(widgets.VBox([prof_dd, widgets.HBox([s_topn, s_ttl]), widgets.HBox([s_maxn, s_spread]), widgets.HBox([i_latency, f_lev]), apply_btn, out]))


## Optuna: Multi-objective Study (maximize EΠ_after_TCA, minimize CVaR95)

In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб запустити оптимізацію (Optuna).
# Це займе кілька хвилин. Після завершення дивись файли у папці reports: optuna_frontier.png, optuna_best_local.json, aurora_suggested.toml.
# Define a search space guided by schema/defaults (best-effort if keys exist)
space = {
    'sizing.limits.max_notional_usd': (100.0, 20000.0, 'float'),
    'sizing.limits.leverage_max': (1.0, 5.0, 'float'),
    'execution.sla.max_latency_ms': (10, 80, 'int'),
    'execution.router.spread_limit_bps': (10.0, 120.0, 'float'),
    'tca.adverse_window_s': (0.5, 5.0, 'float'),
    'universe.ranking.top_n': (3, 50, 'int'),
    'reward.ttl_minutes': (15, 240, 'int'),
}

def get_series_is(df):
    if 'is_bps' in df.columns:
        return pd.to_numeric(df['is_bps'], errors='coerce').dropna()
    if 'slippage_bps' in df.columns:
        return -pd.to_numeric(df['slippage_bps'], errors='coerce').dropna()
    return pd.Series(dtype=float)

def evaluate_objectives(cfg: dict, df: pd.DataFrame):
    # EΠ_after_TCA proxy = mean(IS_bps - fees - impact) if present, else mean(IS_bps)
    is_s = get_series_is(df)
    if is_s.empty:
        e_pi = -1e6
        cvar95 = 1e6
        return e_pi, cvar95
    fees = pd.to_numeric(df.get('fees_bps', 0.0), errors='coerce').fillna(0.0)
    impact = pd.to_numeric(df.get('impact_bps', 0.0), errors='coerce').fillna(0.0)
    e_pi = float((is_s - fees[:len(is_s)].values - impact[:len(is_s)].values).mean())
    # CVaR95 on IS series downside (positive magnitude)
    x = is_s.values.astype(float)
    if len(x) >= 20:
        q = np.quantile(x, 0.05)
        tail = x[x<=q]
        cvar95 = -float(tail.mean())
    else:
        cvar95 = 1e6
    return e_pi, cvar95

def suggest_and_overlay(trial, base_cfg: dict):
    chosen = {}
    for key, (lo, hi, typ) in space.items():
        if typ=='int':
            val = trial.suggest_int(key, int(lo), int(hi))
        else:
            val = trial.suggest_float(key, float(lo), float(hi))
        # set nested
        parts = key.split('.')
        cur = chosen
        for p in parts[:-1]:
            cur = cur.setdefault(p, {})
        cur[parts[-1]] = val
    return deep_merge(base_cfg, chosen), chosen

# Multi-objective study
study = optuna.create_study(directions=['maximize','minimize'], study_name='aurora_multiobj')
N_TRIALS = 30  # adjust as needed

def objective(trial):
    cfg_try, chosen = suggest_and_overlay(trial, effective_cfg)
    e_pi, cvar95 = evaluate_objectives(cfg_try, df_logs)
    trial.set_user_attr('params_nested', chosen)
    trial.set_user_attr('EPI_after_TCA', e_pi)
    trial.set_user_attr('CVaR95', cvar95)
    return e_pi, cvar95

study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=False)
print('Trials complete:', len(study.trials))
# Extract Pareto front
front = optuna.visualization.plot_pareto_front(study) if px else None
# Save matplotlib fallback if plotly not available
pts = [(t.values[0], t.values[1]) for t in study.trials if t.values is not None]
plt.figure(figsize=(5,4))
if pts:
    xs, ys = zip(*pts)
    plt.scatter(xs, ys, s=14, alpha=0.7)
plt.xlabel('EΠ_after_TCA (bps) — maximize')
plt.ylabel('CVaR95 (bps) — minimize')
plt.title('Optuna Pareto Frontier')
plt.tight_layout()
out_png = REPORTS / 'optuna_frontier.png'
plt.savefig(out_png, dpi=160)
print('Saved pareto front to', out_png)

# Save top configurations (by hypervolume rank or simply EPI descending, CVaR asc heuristic)
trials_sorted = sorted(study.trials, key=lambda t: (-(t.values[0] if t.values else -1e9), (t.values[1] if t.values else 1e9)))
top5 = trials_sorted[:5]
best_payload = []
for t in top5:
    best_payload.append({
        'EPI_after_TCA': t.user_attrs.get('EPI_after_TCA'),
        'CVaR95': t.user_attrs.get('CVaR95'),
        'params': t.user_attrs.get('params_nested')
    })
best_json = REPORTS / 'optuna_best_local.json'
best_json.write_text(json.dumps(best_payload, indent=2), encoding='utf-8')
print('Saved top configs to', best_json)

# Freeze-suggest a TOML config overlay using the best trial (index 0)
if top5:
    best_cfg = deep_merge(effective_cfg, top5[0].user_attrs.get('params_nested') or {})
    (REPORTS / 'aurora_suggested.toml').write_text(dump_toml(best_cfg), encoding='utf-8')
    print('Wrote freeze config:', REPORTS / 'aurora_suggested.toml')


## OOS Check: Load another day/symbol and re-evaluate top-5

In [ ]:
# ІНСТРУКЦІЯ: Натисни ▶, щоб перевірити на інших даних (OOS).
# У Colab завантаж інші файли *.jsonl.gz (інший день чи символ). Внизу з'явиться таблиця з оцінками для топ-5 конфігів.
# Load OOS logs
OOS_FILES = []
if IN_COLAB and not OOS_FILES:
    try:
        from google.colab import files
        print('Upload OOS log files (*.jsonl.gz) ...')
        up2 = files.upload()
        OOS_FILES = [Path(k) for k in up2.keys()]
    except Exception:
        pass
df_oos = load_logs_to_df(OOS_FILES)
print('OOS rows:', len(df_oos))

# Re-evaluate top-5 configs saved earlier
best_json = REPORTS / 'optuna_best_local.json'
if best_json.exists() and len(df_oos)>0:
    top = json.loads(best_json.read_text(encoding='utf-8'))
    results = []
    for item in top:
        overlay = item.get('params') or {}
        cfg_try = deep_merge(effective_cfg, overlay)
        epi, cvar = evaluate_objectives(cfg_try, df_oos)
        results.append({'EPI_after_TCA_OOS': epi, 'CVaR95_OOS': cvar, 'params': overlay})
    display(pd.DataFrame(results))
else:
    print('No top-5 results found or OOS logs empty. Run Optuna and provide OOS logs.')


## Generate Logs (Historical/Synthetic)

ІНСТРУКЦІЇ:
- Якщо маєш історичні дані і можеш запустити Aurora симуляцію — обери метод 'aurora_sim'.
- Якщо ні — обери 'synthetic' і згенеруй синтетичні логи на ~цільовий обсяг (наприклад, 1000 МБ ≈ 1 ГБ).
- Файли збережуться до `/content/drive/MyDrive/datasets/` і будуть підхоплені секцією Load Logs.


In [ ]:
from pathlib import Path as _P
import gzip, json, os, random, time
import math

OUT_DIR = _P('/content/drive/MyDrive/datasets') if IN_COLAB else (ROOT / 'datasets')
OUT_DIR.mkdir(parents=True, exist_ok=True)
METHOD = 'synthetic'  # 'synthetic' or 'aurora_sim'
TARGET_MB = 1000       # приблизно 1 ГБ стиснених логів (може відрізнятись)
SEED = 123
random.seed(SEED)

def _rand_choice(p_true=0.5):
    return 'maker' if random.random() < p_true else 'taker'

def gen_synthetic_logs(target_mb: int, out_path: _P):
    # Створюємо gzip-потік і пишемо JSONL рядки поки файл не досягне ~target_mb
    n = 0
    start = time.time()
    with gzip.open(out_path, 'wt', encoding='utf-8') as f:
        size_mb = 0.0
        while True:
            cid = f'c{n}'
            ts_ms = int(1_710_000_000_000 + n*50)
            qty = max(1.0, random.gauss(1.5, 0.5))
            filled = qty if random.random() < 0.8 else random.random()*qty
            slip = random.gauss(0.0, 3.0)
            is_bps = slip * (-1.0) + random.gauss(1.0, 1.0)
            fees = abs(random.gauss(0.5, 0.2))
            impact = abs(random.gauss(0.7, 0.3))
            spread = abs(random.gauss(20.0, 10.0))
            rec = {
                'event_code': random.choice(['ORDER.SUBMIT','ORDER.ACK','ORDER.FILL','ORDER.CANCEL']),
                'cid': cid,
                'ts_ms': ts_ms,
                'order_qty': qty,
                'filled_qty': filled,
                'slippage_bps': slip,
                'is_bps': is_bps,
                'fees_bps': fees,
                'impact_bps': impact,
                'spread_bps': spread,
                'liquidity': _rand_choice(0.6),
            }
            if random.random() < 0.05:
                rec['reason_code'] = random.choice(['SPREAD_GUARD','VOL_GUARD','CVAR_GUARD','LATENCY_GUARD'])
            f.write(json.dumps(rec)+'
')
            n += 1
            if n % 5000 == 0:
                f.flush()
                try:
                    size_mb = os.path.getsize(out_path) / (1024*1024.0)
                except Exception:
                    size_mb = 0.0
                if size_mb >= target_mb:
                    break
    print(f'Generated ~{size_mb:.1f} MB compressed, {n} rows, took {time.time()-start:.1f}s -> {out_path}')

def gen_with_aurora_sim(target_mb: int, out_dir: _P):
    # Спробувати викликати інструменти з репо для локального сімулятора
    # (потрібен повний репозиторій у Colab)
    import subprocess, sys
    # Базово: запустити dryrun, який генерує JSONL у logs/, далі стиснути і перемістити у datasets/
    try:
        rc = subprocess.run([sys.executable, str(ROOT/'tools'/'sim_local_dryrun.py')], check=False)
        print('sim_local_dryrun exit code:', rc.returncode)
    except Exception as e:
        print('Aurora sim failed, falling back to synthetic. Error:', e)
        return gen_synthetic_logs(target_mb, out_dir/('synth_'+str(int(time.time()))+'.jsonl.gz'))
    # Знайти логи у ROOT/logs та стиснути у один файл (приблизно цільовий розмір)
    parts = sorted((ROOT/'logs').glob('*.jsonl'))
    out_path = out_dir/('aurora_sim_'+str(int(time.time()))+'.jsonl.gz')
    rows = 0
    size_mb = 0.0
    with gzip.open(out_path, 'wt', encoding='utf-8') as gz:
        for p in parts:
            for line in p.read_text(encoding='utf-8').splitlines():
                gz.write(line+'
')
                rows += 1
                if rows % 10000 == 0:
                    gz.flush()
                    size_mb = os.path.getsize(out_path)/(1024*1024.0)
                    if size_mb >= target_mb:
                        break
            if size_mb >= target_mb:
                break
    print(f'Packed ~{size_mb:.1f} MB from Aurora logs into {out_path} (rows={rows})')

# Run generation
if METHOD == 'synthetic':
    out = OUT_DIR/('synth_'+str(int(time.time()))+'.jsonl.gz')
    gen_synthetic_logs(TARGET_MB, out)
else:
    gen_with_aurora_sim(TARGET_MB, OUT_DIR)


## Decision Audit (All Metrics & Parameters)

ІНСТРУКЦІЇ:
- Натисни ▶ у наступній комірці — ми зберемо всі ключові метрики і параметри рішень.
- Результати збережуться як: `/content/drive/MyDrive/reports/decision_factors.json` і `decision_audit.md`.


In [ ]:
# Collect decision parameters + computed metrics into a single report
def pick_keys(d, paths):
    out = {}
    for p in paths:
        out[p] = deep_get(d, p)
    return out

cfg_paths = [
  'risk.limits.dd_day_bps','risk.limits.position_usd','risk.limits.cvar_usd',
  'risk.cvar.limit','risk.cvar.alpha','risk.cvar.method','risk.cvar.lookback',
  'execution.sla.max_latency_ms','execution.sla.kappa_bps_per_ms','execution.sla.target_fill_prob',
  'execution.router.mode','execution.router.mode_default','execution.router.spread_limit_bps',
  'execution.fees.default_maker_bps','execution.fees.default_taker_bps',
  'sizing.kelly.risk_aversion','sizing.kelly.clip_max','sizing.portfolio.method','sizing.portfolio.cvar_limit',
  'sizing.limits.min_notional_usd','sizing.limits.max_notional_usd','sizing.limits.leverage_max',
  'reward.ttl_minutes','reward.tp_levels_bps','reward.tp_sizes',
  'tca.adverse_window_s','tca.mark_ref','tca.aggregate_interval_s',
  'features.scaling.method',
  'universe.ranking.wL','universe.ranking.wS','universe.ranking.wP','universe.ranking.wR','universe.ranking.add_thresh','universe.ranking.drop_thresh',
  'order_sink.mode','order_sink.sim_local.latency_ms_range','order_sink.sim_local.slip_bps_range'
]
cfg_selected = pick_keys(effective_cfg if 'effective_cfg' in globals() else CFG, cfg_paths)

# Reuse metrics from earlier cell or recompute if needed
if 'metrics' not in globals() or not metrics:
    metrics, ref_series = compute_metrics(df_logs)

audit = {
  'config': cfg_selected,
  'metrics': metrics,
}
# Simple correctness checks (heuristics)
epi = metrics.get('tca',{}).get('IS_bps')
cvar = metrics.get('risk',{}).get('CVaR95')
sla_max = deep_get(effective_cfg if 'effective_cfg' in globals() else CFG, 'execution.sla.max_latency_ms', 25)
sla_p50 = metrics.get('order_level',{}).get('latency_ms',{}).get('submit_ack',{}).get('p50')
notes = []
if epi is not None:
    notes.append(f'EPI_after_TCA ~ {epi:.2f} bps (higher is better)')
if cvar is not None:
    notes.append(f'CVaR95 ~ {cvar:.2f} bps (lower is better)')
try:
    if sla_p50 is not None and sla_p50 > float(sla_max):
        notes.append(f'Warning: latency p50 {sla_p50:.1f}ms exceeds SLA {sla_max}ms')
except Exception:
    pass
audit['notes'] = notes

# Save
(REPORTS/'decision_factors.json').write_text(json.dumps(audit, ensure_ascii=False, indent=2), encoding='utf-8')
md = ['# Decision Audit', '', '## Notes'] + ['- '+n for n in notes]
(REPORTS/'decision_audit.md').write_text('
'.join(md), encoding='utf-8')
print('Wrote:', REPORTS/'decision_factors.json')
print('Wrote:', REPORTS/'decision_audit.md')
